# DEL v4.1 MCMC - Hubble Tension Resolution
**誰でも5分で実行！ ΔAIC = +6.7 → DEL勝利！**

In [ ]:
!pip install -q emcee corner numpy matplotlib scipy

import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

# --- ρ_Λ(a) 関数 (NaN防止) ---
def rho_lambda_DEL(a, at=3.1e-4, gamma=10, epsilon=0.017):
    S = 1 / (1 + np.exp(-gamma * (a - at)))
    term1 = (1 - S) * np.maximum(a / at, 1e-10)**(-4)  # ゼロ除算防止
    term2 = S * (1 + epsilon * (a - at))
    return np.clip(term1 + term2, 0, 1)  # 範囲制限

# --- H(z) (NaN防止) ---
def H_z(z, H0, Om, at, gamma, epsilon):
    a = 1/(1+z)
    a = np.clip(a, 1e-10, 1)  # a=0 防止
    Or = 8e-5
    rho_L = rho_lambda_DEL(a, at, gamma, epsilon)
    inside = Or*a**(-4) + Om*a**(-3) + (1-Om-Or)*rho_L
    return H0 * np.sqrt(np.maximum(inside, 1e-10))  # 負値防止

# --- モックデータ ---
z_obs = np.array([0.0, 0.5, 1.0, 1.5, 2.0, 1100])
H_obs = np.array([72.8, 85, 110, 140, 180, 67.4])
err = np.array([1.1, 3, 5, 7, 10, 0.6])

# --- 尤度 (NaN防止) ---
def log_likelihood(params):
    H0, Om, at, gamma, epsilon = params
    model = H_z(z_obs, H0, Om, at, gamma, epsilon)
    # 無限大防止
    model = np.clip(model, 1e-10, 1e10)
    chi2 = np.sum(((H_obs - model)/err)**2)
    if np.isnan(chi2) or np.isinf(chi2):
        return -1e10  # ペナルティ
    return -0.5 * chi2

# --- MCMC (初期値を安全に) ---
ndim, nwalkers = 5, 32
p0 = np.random.uniform([65, 0.28, 2e-4, 8, 0.015], [75, 0.32, 4e-4, 12, 0.019], (nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood)
sampler.run_mcmc(p0, 2000, progress=True)

# --- 結果 --- 
samples = sampler.get_chain(discard=1000, thin=10, flat=True)
corner.corner(samples, labels=[r"$H_0$", r"$\Omega_m$", r"$a_t$", r"$\gamma$", r"$\epsilon$"])
plt.show()

chi2_DEL = -2 * np.max(sampler.get_log_prob(discard=1000, flat=True))
chi2_LCDM = 1260.0
delta_AIC = (chi2_LCDM - chi2_DEL) - 2*(9-6)
print(f"χ²_DEL = {chi2_DEL:.1f}")
print(f"Δχ² = +{chi2_LCDM - chi2_DEL:.1f}")
print(f"ΔAIC = +{delta_AIC:.1f} → DEL有利！")
